In [1]:
import pandas as pd
import heapq

In [2]:
def knn(k, filename='wdbc.data.mb.csv'):
    predictions_results = []
    dataframe = pd.read_csv(filename, header=None)
    df_list = dataframe.values.tolist()
    test_set, training_set = split_sets(df_list, 0.7)

    if not test_set or not training_set:
        return []

    for i in range(len(test_set)):
        test_item = test_set[i]
        distances_heap = []

        for j in range(len(training_set)):
            training_item = training_set[j]
            dist_value = compute_dist(test_item, training_item)
            heapq.heappush(distances_heap, (dist_value, training_item))

        neighbors = heapq.nsmallest(k, distances_heap)

        malignant_count = 0
        benign_count = 0
        for dist, neighbor_item in neighbors:
            if neighbor_item[-1] == 1:
                malignant_count += 1
            else:
                benign_count += 1

        classification = 1 if malignant_count >= benign_count else -1

        actual_label = test_item[-1]
        predictions_results.append(actual_label == classification)

    return predictions_results